## *Passing Authority: Multisig accounts and Rekeying
#### 03.5 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2023-02-08 (started 2022-02-02)

- Create and use k-out-of-n multisignature accounts
- Rekey an account

## Setup
Starting with chapter 3.4, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [81]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [82]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn, MultisigTransaction
import algosdk.error
import json

In [83]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])
print(Dina['public'])

33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA
GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4
MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM
ZFTSGNBBFDH544IDUVB7S4B67CX6PJL6MMFJYT3GGDH2GWJ4N7H6VKRU2Y
OH6I6P7ZFKG6I5BBHB7R6C7BLKVCRAJUV5QTR3BZQZINPLXTWD3B4MVEKA


### Connecting to the Algorand testnet

In [84]:
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()["last-round"]

27730226

## Multisig account
Alice, Bob and Charlie create a multisig account, 
with the requirement that at least 2 out of 3 have to sign.

#### Step 1: Create the Multisig account
* Using the new command `Multisig`

In [85]:
from algosdk.transaction import Multisig

version   = 1     # multisig version, currently always 1
threshold = 2     # min 2 signatures required
account_list = [Alice['public'], Bob['public'], Charlie['public']]

msig = Multisig(version, threshold, account_list)
msig_address = msig.address()

print("Multisig Address: ", msig_address)

Multisig Address:  D7ZXNZZYMHILUF32BCCWGAAF6PT665Y3UKW5Q3FAL2J5YP3FRIXOL5G2HY


#### Step 2: Fund the multisig account
* Can be funded via PaymentTxn
* Here we fund from a dispenser

In [86]:
print('https://dispenser.testnet.aws.algodev.network/?account='+msig_address)

https://dispenser.testnet.aws.algodev.network/?account=D7ZXNZZYMHILUF32BCCWGAAF6PT665Y3UKW5Q3FAL2J5YP3FRIXOL5G2HY


#### Step 3.1: Create a simple payment transaction
* Together, they want to send 1.2 ALGOs to Dina

In [88]:
sp = algod_client.suggested_params()
amt = int(1.2*1E6)

txn = PaymentTxn(sender=msig_address,         # <-- From Multisig address
                 sp=sp, 
                 receiver=Dina['public'], 
                 amt = amt
                )

#### Step 3.2: Convert simple transaction to a multisig transaction and sign it
* Alice and Bob sign the transaction

In [89]:
# Create multisig transaction
mtx = MultisigTransaction(txn, msig)

# Collect at least 2 signatures
mtx.sign(Alice['private'])
mtx.sign(Bob['private'])

#### Step 3.3: Send the transaction

In [91]:
txid = algod_client.send_transaction(mtx)

#### Step 3.4: Wait for confirmation

In [92]:
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27730241.
Waiting for round 27730241 to finish.
Waiting for round 27730242 to finish.
Transaction 7PNOHOUJ75AORJ4OWE5TQLPVEVQTHLXRQJQ27BRJZTGXB2CJOXOQ confirmed in round 27730243.


#### Check Dina's holdings

In [93]:
algod_client.account_info(Bob["public"])["amount"] / 1e6


3.793

#### Exercise
* Repeat steps 3.1-3.4 with Alice and Charlie signing the transaction
* Repeat steps 3.1-3.4 with Alice and Dina signing 
* Repeat steps 3.1-3.4 with only Alice signing

## Rekeying
Case 1
* Imagine your private key has been compromised
* You have lots of assets and maybe ongoing smart contracts with this address
* So you want to "change the password"

Case 2
* Bob is very busy and wants Charlie to sign for him

#### Bob rekeys to Charlie
Rekeying is done via a simple payment transaction of 0 ALGOs, that has an additional rekey entry

In [94]:
# Step 1: prepare special transaction
sp = algod_client.suggested_params()
amt = 0                                          # <-- Send zero ALGOs

txn = PaymentTxn(sender = Bob['public'],         # <-- Bob wants to rekey
                 sp = sp, 
                 receiver = Bob['public'],       # <-- Bob wants to rekey
                 amt = amt,
                 rekey_to = Charlie['public']    # <-- Charlie should be able to sign for Bob
                )

In [95]:
# Steps 2+3: sign and send
stxn = txn.sign(Bob['private'])                 # <-- Bob signs one last time
txid = algod_client.send_transaction(stxn)

In [96]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27730248.
Waiting for round 27730248 to finish.
Waiting for round 27730249 to finish.
Transaction BJ2BMX7F5NHQK3LODRP45IMXPGQBACXHKHB2N3LME7ZSUOH2L72A confirmed in round 27730250.


#### Charlie signs for Bob
* Charlie makes a payment of 1.4 ALGO to Alice out of Bob's account

In [97]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(1.4*1E6)                                

txn = PaymentTxn(sender = Bob['public'],           # <-- From Bob
                 sp = sp, 
                 receiver = Alice['public'],       # <-- To Alice 
                 amt = amt
                 )

# Steps 2+3: sign and send
stxn = txn.sign(Charlie['private'])                # <-- Charlie now signs for Bob
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27730250.
Waiting for round 27730250 to finish.
Waiting for round 27730251 to finish.
Transaction K33KATZ6NOOBOSDSVIJPTVZ3ATVLMRAHGWWTZCS4TGQHV5EIC4OQ confirmed in round 27730252.


#### Bob cannot sign for himself
* Bob wants to make a payment of 1.7 ALGO to Dina

In [100]:
# Step 1: prepare special transaction

sp = algod_client.suggested_params()
amt = int(1.7*1E6)                                

txn = PaymentTxn(sender = Bob['public'],         # <-- From Bob
                 sp = sp, 
                 receiver = Dina['public'],       # <-- To Dina 
                 amt = amt
                )

# Steps 2+3: sign and send
stxn = txn.sign(Bob['private'])                 # <-- Bob tries to sign
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

AlgodHTTPError: TransactionPool.Remember: transaction ZCTTWOFMRKNLBXMTXJG2QLO25FNOW2H45LCVQBTP3DX5ZA65Y3DQ: should have been authorized by ZFTSGNBBFDH544IDUVB7S4B67CX6PJL6MMFJYT3GGDH2GWJ4N7H6VKRU2Y but was actually authorized by MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM

#### ❗️Charlie rekeys back to Bob
* Charlie rekeys the Bob account back to Bob
* **Important step for the rest of the course**

In [101]:
# Step 1: prepare special transaction
sp = algod_client.suggested_params()
amt = 0                                          # <-- Rekeying = send 0 ALGOs

txn = PaymentTxn(sender = Bob['public'],         # <-- The address which is rekeyed
                 sp = sp, 
                 receiver = Bob['public'],       # <-- The address which is rekeyed 
                 amt = amt,
                 rekey_to = Bob['public']        # <-- Bob should (again) be able to sign for himself
                )

In [103]:
# Steps 2+3: sign and send
stxn = txn.sign(Charlie['private'])              # <-- Charlie signs the rekeying
txid = algod_client.send_transaction(stxn)

In [104]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27730270.
Waiting for round 27730270 to finish.
Waiting for round 27730271 to finish.
Transaction VOM4XIGOMZITEHFQQDZYMILEKLFA4C5HE4FY4U4TXNJGOESAM2BA confirmed in round 27730272.


#### Exercise
* To test the rekeying, run the payment transactions above again
    * Charlie cannot sign for Bob anymore
    * Bob can sign again for himself

## ❗️Things that can and will go wrong
* Be sure if you trust the new signature holder
* You cannot rekey back to yourself
* **Rekeying can be easily slipped into a payment transaction**
* Wrong address or typo in rekey means you loose all your money
    * The protocol does not check whether you or someone has actually access to the rekey address